October 31, 2023

Running this example on my brand spanking new 4090!

[How to Load LLama 13b for Inference on a single RTX 4090](https://codehammer.io/how-to-load-llama-13b-for-inference-on-a-single-rtx-4090/)

And no, this notebook is NOT part of the repo ...

[Natural Language Processing with Transformers by Lewis Tunstall, Leandro von Werra, Thomas Wolf.](https://github.com/nlp-with-transformers/notebooks)

docker container start hfpt_Oct28

To copy downloaded models from this docker container to some external source, you need to know the id of this container plus the location of the files. For these Meta LLMs, the source folder is '/home/rob/Data2/huggingface/transformers' and the container id is 'c9b676310ea0'.

So to copy a model from this folder, drop into a terminal window and type 'ls /home/rob/Data2/huggingface/transformers' to see what is available, then from an external terminal window and with the container running (actually it doesn't need to be running!), you can type 'docker cp c9b676310ea0:/home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-chat-hf /home/rob/Data3/huggingface/transformers'


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# So turns out this no longer exists!
checkpoint = "decapoda-research/llama-13b-hf"

# And there is now LLama2 ...
checkpoint = "meta-llama/Llama-2-7b-hf"
checkpoint = "meta-llama/Llama-2-13b-chat-hf"
checkpoint = "meta-llama/Llama-2-7b-chat-hf" # This has been downloaded.


In [3]:
%%time
# model = AutoModelForCausalLM.from_pretrained(checkpoint) # takes up 29.4gb of ram
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True) # wow! this only takes up 3.4gb of ram and 7660Mib of VRAM!
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True, device_map='auto') # hmm this fails
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 4.27 s, sys: 2.34 s, total: 6.61 s
Wall time: 5.58 s


In [2]:
%%time
# how about we try the 13b model?
checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True) # Holy crap, this works! 13,948MiB of VRAM!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 5.81 s, sys: 7.7 s, total: 13.5 s
Wall time: 15.8 s


In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

CPU times: user 46.7 ms, sys: 1.5 ms, total: 48.2 ms
Wall time: 122 ms


These next 3 cells you should keep them, but don't run them ... this dumps the model into these 3 locally container folders under /home/rob/Data3.

NOTHING gets written into the external '/home/rob/Data3' folder ... !

I think I need to gain a much better understanding about docker volumes ...

In [6]:
# WOW ... this does NOT save to the external Data3 drive, but saves it again into this container, creating a folder with the same name!
# model.save_pretrained('/home/rob/Data3/huggingface')

In [7]:
# model.save_pretrained('/home/rob/Data3/fuckthisshit')

In [8]:
# model.save_pretrained('/home/rob/Data3/blowme')

In [4]:
# hmmm this does not fit on the 4090! ... damn!
model.to("cuda")

You shouldn't move a model when it is dispatched on multiple devices.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [2]:
%%time
checkpoint = "meta-llama/Llama-2-7b" 
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# OSError: meta-llama/Llama-2-7b does not appear to have a file named config.json. Checkout 'https://huggingface.co/meta-llama/Llama-2-7b/main' for available files.

OSError: meta-llama/Llama-2-7b does not appear to have a file named config.json. Checkout 'https://huggingface.co/meta-llama/Llama-2-7b/main' for available files.

In [ ]:
%%time
checkpoint = "meta-llama/Llama-2-13b" # Nope!
checkpoint = "meta-llama/Llama-2-13b-chat-hf" # Takes up all 32gb of ram plus 24gb of swap space!
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
type(model)

In [ ]:
model.to("cuda")